<a href="https://colab.research.google.com/github/CodingAW/C140/blob/main/C138.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
u = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata

 56% 5.00M/8.89M [00:00<00:00, 13.1MB/s]
100% 8.89M/8.89M [00:00<00:00, 20.0MB/s]


In [5]:
!unzip tmdb-movie-metadata.zip

Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [6]:
import pandas as pd
import numpy as py
import plotly.express as px

In [7]:
df1 = pd.read_csv("tmdb_5000_credits.csv")
df2 = pd.read_csv("tmdb_5000_movies.csv")

df1.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [8]:
df2.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [9]:
df1.columns = ['id', 'title', 'cast', 'crew']
df2 = df2.merge(df1, on= "id")
df2.head(6)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-01,890871626,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso

In [10]:
# C139

C = df2["vote_average"].mean()
print(C)

6.092171559442011


In [11]:
m = df2['vote_count'].quantile(0.9)
print(m)

1838.4000000000015


In [12]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]
print(q_movies.shape)

(481, 23)


In [13]:
def weighted_rating(x, m= m, C= C):
  v = x["vote_count"]
  R = x['vote_average']
  return ((v/(v+m))*R)+((m/(v+m))*C)

q_movies['score'] = q_movies.apply(weighted_rating, axis= 1)


In [14]:
q_movies = q_movies.sort_values("score", ascending= False)
q_movies[['original_title', 'vote_count', 'vote_average', 'score']].head(10)

,original_title,vote_count,vote_average,score
1881,The Shawshank Redemption,8205,8.5,8.059258
662,Fight Club,9413,8.3,7.939256
65,The Dark Knight,12002,8.2,7.920020
3232,Pulp Fiction,8428,8.3,7.904645
96,Inception,13752,8.1,7.863239
3337,The Godfather,5893,8.4,7.851236
95,Interstellar,10867,8.1,7.809479
809,Forrest Gump,7927,8.2,7.803188
329,The Lord of the Rings: The Return of the King,8064,8.1,7.727243
1990,The Empire Strikes Back,5879,8.2,7.697884


In [15]:
fig = px.bar(q_movies.head(10).sort_values("score", ascending= False), x="score", y="original_title", orientation="h")
fig.show()

In [17]:
# C140

df2[["original_title", "cast", "crew", "keywords", "genres"]].head(6)

,original_title,cast,crew,keywords,genres
0,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
3,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam..."
4,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
5,Spider-Man 3,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de...","[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...","[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na..."


In [35]:
from ast import literal_eval
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
features = ["cast", "crew", "keywords", "genres"]

for feature in features:
  df2[feature] = df2[feature].apply(literal_eval)

df2.dtypes

budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title_x                  object
vote_average            float64
vote_count                int64
title_y                  object
cast                     object
crew                     object
dtype: object

In [23]:
def get_director(x):
  for i in x:
    if i["job"] == "Director":
      return i["name"]
  return np.nan

df2["director"] = df2["crew"].apply(get_director)

In [25]:
def get_list(x):
  if isinstance(x, list):
    names = [i["name"] for i in x]
    return names
  return []

features = ["cast", "keywords", "genres"]

for feature in features:
  df2[feature] = df2[feature].apply(get_list)

In [26]:
df2[["original_title", "cast", "keywords", "genres", 'director']].head(6)

,original_title,cast,keywords,genres,director
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]",James Cameron
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]",Gore Verbinski
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]",Sam Mendes
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]",Christopher Nolan
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]",Andrew Stanton
5,Spider-Man 3,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[dual identity, amnesia, sandstorm, love of on...","[Fantasy, Action, Adventure]",Sam Raimi


In [28]:
def clean_data(x):
  if isinstance(x, list):
    return [str.lower(i.replace(" ", "")) for i in x]
  else:
    if isinstance(x, str):
      return str.lower(x.replace(" ", ""))
    else:
      return ""

features = ["cast", "keywords", "genres", "director"]

for feature in features:
  df2[feature] = df2[feature].apply(clean_data)

In [29]:
df2[["original_title", "cast", "keywords", "genres", 'director']].head(6)

,original_title,cast,keywords,genres,director
0,Avatar,"[samworthington, zoesaldana, sigourneyweaver, ...","[cultureclash, future, spacewar, spacecolony, ...","[action, adventure, fantasy, sciencefiction]",jamescameron
1,Pirates of the Caribbean: At World's End,"[johnnydepp, orlandobloom, keiraknightley, ste...","[ocean, drugabuse, exoticisland, eastindiatrad...","[adventure, fantasy, action]",goreverbinski
2,Spectre,"[danielcraig, christophwaltz, léaseydoux, ralp...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[action, adventure, crime]",sammendes
3,The Dark Knight Rises,"[christianbale, michaelcaine, garyoldman, anne...","[dccomics, crimefighter, terrorist, secretiden...","[action, crime, drama, thriller]",christophernolan
4,John Carter,"[taylorkitsch, lynncollins, samanthamorton, wi...","[basedonnovel, mars, medallion, spacetravel, p...","[action, adventure, sciencefiction]",andrewstanton
5,Spider-Man 3,"[tobeymaguire, kirstendunst, jamesfranco, thom...","[dualidentity, amnesia, sandstorm, loveofone's...","[fantasy, action, adventure]",samraimi


In [30]:
a = ["hello", "how", "are", "you"]
" ".join(a)

'hello how are you'

In [31]:
def create_soup(x):
  return " ".join(x['keywords']) + " " + " ".join(x["cast"]) + " " + " ".join(x["director"]) + " " + " ".join(x["genres"])

df2['soup'] = df2.apply(create_soup, axis=1)

In [32]:
df2[["original_title", "cast", "keywords", "genres", 'director', 'soup']].head(6)

,original_title,cast,keywords,genres,director,soup
0,Avatar,"[samworthington, zoesaldana, sigourneyweaver, ...","[cultureclash, future, spacewar, spacecolony, ...","[action, adventure, fantasy, sciencefiction]",jamescameron,cultureclash future spacewar spacecolony socie...
1,Pirates of the Caribbean: At World's End,"[johnnydepp, orlandobloom, keiraknightley, ste...","[ocean, drugabuse, exoticisland, eastindiatrad...","[adventure, fantasy, action]",goreverbinski,ocean drugabuse exoticisland eastindiatradingc...
2,Spectre,"[danielcraig, christophwaltz, léaseydoux, ralp...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[action, adventure, crime]",sammendes,spy basedonnovel secretagent sequel mi6 britis...
3,The Dark Knight Rises,"[christianbale, michaelcaine, garyoldman, anne...","[dccomics, crimefighter, terrorist, secretiden...","[action, crime, drama, thriller]",christophernolan,dccomics crimefighter terrorist secretidentity...
4,John Carter,"[taylorkitsch, lynncollins, samanthamorton, wi...","[basedonnovel, mars, medallion, spacetravel, p...","[action, adventure, sciencefiction]",andrewstanton,basedonnovel mars medallion spacetravel prince...
5,Spider-Man 3,"[tobeymaguire, kirstendunst, jamesfranco, thom...","[dualidentity, amnesia, sandstorm, loveofone's...","[fantasy, action, adventure]",samraimi,dualidentity amnesia sandstorm loveofone'slife...


In [37]:
count = CountVectorizer(stop_words= "english")
count_matrix = count.fit_transform(df2['soup'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
df2 = df2.reset_index()
indices = pd.Series(df2.index, index= df2['original_title'])

In [38]:
def get_reccommendations(title, cosine_sim):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))

  sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse= True)
  sim_scores = sim_scores[1:11]
  movie_indices = [i[0] for  i in sim_scores]
  return df2['original_title'].iloc[movie_indices]

get_reccommendations("Avatar", cosine_sim2)

4401                  The Helix... Loaded
1201                            Predators
1932                               Sheena
168     Final Fantasy: The Spirits Within
539                            Titan A.E.
661            Zathura: A Space Adventure
2327                             Predator
260                          Ender's Game
3405           Stargate: The Ark of Truth
3856          In the Name of the King III
Name: original_title, dtype: object

In [39]:
get_reccommendations("Spider-Man 3", cosine_sim2)

30                      Spider-Man 2
159                       Spider-Man
2644                       Ong Bak 2
215     4: Rise of the Silver Surfer
1652            Dragonball Evolution
511                            X-Men
3856     In the Name of the King III
1296                    Superman III
3826                Chain of Command
1932                          Sheena
Name: original_title, dtype: object